# Fish image classification with Keras Continued
 
 Let's digging deeper into Keras!
 
 note :This notebook is inspired by Traffic_Signs_Recognition project in SelfDrivingCarND
 

In [1]:
import os
import random
import cv2


import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Allow image embeding in notebook
%matplotlib inline

## Load the Data

starting by Loading the Data


In [2]:
def load_train_data(data_dir):
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir)
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []

    category = 0
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f)
                      for f in os.listdir(label_dir)
                      if f.endswith(".jpg")]
        
       
        for f in file_names:
            img = cv2.imread(f)
            imresize = cv2.resize(img, (200, 125))
            #plt.imshow(imresize)
            images.append(imresize)
            labels.append(category)
        category += 1
        
    return images, labels



In [3]:
train_data_dir = os.path.join('/home/suh/바탕화면/train')
images, labels = load_train_data(train_data_dir)

In [4]:

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (32, 32), interpolation=cv2.INTER_LINEAR)
    return resized

def load_test_data():
    import glob
    path = os.path.join( '/home/suh/바탕화면/test_stg1/*.jpg')
    files = sorted(glob.glob(path))

    test_images = []
    test_labels = []
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl)
        test_images.append(img)
        test_labels.append(flbase)

    return test_images, test_labels

In [5]:

test_images,test_labels = load_test_data()

In [7]:
X_train, y_train = images, labels
X_test, y_test = test_images, test_labels

Do some data summary

In [8]:
n_train = len(X_train)
n_test = len(X_test)
image_shape = X_train[0].shape
n_classes = len(set(y_train))

print("Number of training example: {}".format(n_train))
print("Number of testing examples: {} ".format(n_test))
print("Image data shape: {}".format(image_shape)) 
print("Number of classes: {}".format(n_classes))



Number of training example: 3777
Number of testing examples: 1000 
Image data shape: (125, 200, 3)
Number of classes: 8


## Validate the Network
Split the training data into a training and validation set.


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=432422)

In [10]:
X_train[0].shape

(125, 200, 3)

In [12]:
X_train = np.array(X_train).astype('float32')
X_val =np.array(X_val).astype('float32')
X_train = X_train / 255 - 0.5
X_val = X_val / 255 - 0.5
print(X_train.shape)

(3210, 125, 200, 3)


## Build a Two-Layer Feedfoward Network
The code which is written so far is for data processing, not specific to Keras. thus, let's build Keras-specific code. Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer.

In [13]:
#assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (125,200,3)), "The dimensions of the images are not 125 x 200x 3."
#assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (125,200,3)), "The dimensions of the images are not 125 x 200 x 3."

In [14]:
import math
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

In [15]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers import Dense, Input, Activation

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(125*200*3,)))
model.add(Dense(43, activation='softmax'))

Using TensorFlow backend.


In [16]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 3210))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

AssertionError: 

In [17]:
# Debugging
for l in model.layers:
    print(l.name, l.input_shape, l.output_shape, l.activation)

dense_1 (None, 75000) (None, 128) <function relu at 0x7f0f0e784510>
dense_2 (None, 128) (None, 43) <function softmax at 0x7f0f0e7842f0>


## Train the Network
Compile and train the network for 2 epochs. Use the adam optimizer, with categorical_crossentropy loss.

In [18]:
# TODO: Compile and train the model here.
from keras.utils import np_utils

Y_train = np_utils.to_categorical(y_train, 43)
Y_val = np_utils.to_categorical(y_val, 43)

X_train= X_train.reshape(-1, 125*200*3)
X_val = X_val.reshape(-1, 125*200*3)

model.summary()
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, validation_data=(X_val, Y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 128)           9600128     dense_input_1[0][0]              
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 43)            5547        dense_1[0][0]                    
Total params: 9605675
____________________________________________________________________________________________________
Train on 3210 samples, validate on 567 samples
Epoch 1/20
3210/3210 [==============================] - 15s - loss: 7.1135 - acc: 0.5087 - val_loss: 7.0650 - val_acc: 0.5414
Epoch 2/20
3210/3210 [==============================] - 13s - loss: 6.9117 - acc: 0.5592 - val_loss: 6.7719 - val_acc: 0.5697
Epoch 3/20
3210/3210 [==============================] - 13s - loss: 6.596

In [19]:
print( "The validation accuracy is: %.3f" % history.history['val_acc'][-1] )

The validation accuracy is: 0.732


##  Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 convolutional layer with 32 filters and valid padding.

In [9]:
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=432422)

# TODO: Implement data normalization here.
def normalize_img(image):
    return cv2.normalize(image, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

X_train = np.array([normalize_img(image) for image in X_train], dtype=np.float32)
X_val = np.array([normalize_img(image) for image in X_val], dtype=np.float32)

print(X_train.shape[1:])
layer1_depth = 32
filter_size = 3
num_classes = len(np.unique(y_train))
num_neurons = 128

model = Sequential()
model.add(Convolution2D(layer1_depth, filter_size, filter_size, border_mode='valid', input_shape = (125, 200, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_neurons))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

(125, 200, 3)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 123, 198, 32)  896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 123, 198, 32)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 779328)        0           activation_1[0][0]               
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 128)           99754112    flatten_1[0][0]                  
_____________________________________________________________________________

In [10]:
# TODO: Compile and train the model here.
y_train = np.array(y_train).astype(int)
y_val = np.array(y_val).astype(int)


y_train = np_utils.to_categorical(y_train, nb_classes=num_classes)
y_val = np_utils.to_categorical(y_val, nb_classes=num_classes)

X_train = X_train.reshape(-1, 125, 200, 3)
X_val = X_val.reshape(-1,125, 200, 3)

nb_epoch = 2
batch_size = 32

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

Train on 3210 samples, validate on 567 samples
Epoch 1/2
3210/3210 [==============================] - 1105s - loss: 8.8076 - acc: 0.4480 - val_loss: 8.2438 - val_acc: 0.4885
Epoch 2/2
3210/3210 [==============================] - 981s - loss: 8.8775 - acc: 0.4492 - val_loss: 8.2438 - val_acc: 0.4885


In [11]:

print( "The validation accuracy is: %.3f" % history.history['val_acc'][-1] )

The validation accuracy is: 0.489


## Pooling
Re-construct your network and add a 2x2 pooling layer immediately following your convolutional layer.
Then compile and train the network.

In [24]:

# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=432422)

# TODO: Implement data normalization here.
def normalize_img(image):
    return cv2.normalize(image, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

X_train = np.array([normalize_img(image) for image in X_train], dtype=np.float32)
X_val = np.array([normalize_img(image) for image in X_val], dtype=np.float32)

layer1_depth = 32
filter_size = 5
num_classes = len(np.unique(y_train))
num_neurons = 128

model = Sequential()
model.add(Convolution2D(layer1_depth, filter_size, filter_size, border_mode='valid', input_shape = (125, 200, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(num_neurons))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_2 (Convolution2D)  (None, 121, 196, 32)  2432        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 121, 196, 32)  0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 60, 98, 32)    0           activation_4[0][0]               
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 188160)        0           maxpooling2d_2[0][0]             
___________________________________________________________________________________________

In [12]:
# TODO: Compile and train the model here.
y_train = np.array(y_train).astype(int)
y_val = np.array(y_val).astype(int)

y_train = np_utils.to_categorical(y_train, nb_classes=num_classes)
y_val = np_utils.to_categorical(y_val, nb_classes=num_classes)

X_train = X_train.reshape(-1, 125, 200, 3)
X_val = X_val.reshape(-1, 125, 200, 3)

nb_epoch = 2
batch_size = 32

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

Train on 2728 samples, validate on 482 samples
Epoch 1/2
2728/2728 [==============================] - 201s - loss: 8.3754 - acc: 0.4644 - val_loss: 8.4369 - val_acc: 0.4419
Epoch 2/2
2728/2728 [==============================] - 212s - loss: 2.9668 - acc: 0.6404 - val_loss: 0.6779 - val_acc: 0.7925


In [13]:
print( "The validation accuracy is: %.3f" % history.history['val_acc'][-1] )

The validation accuracy is: 0.793


# Dropout
Re-construct your network and add dropout after the pooling layer. Set the dropout rate to 50%.

In [21]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.


from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=432422)

# TODO: Implement data normalization here.
def normalize_img(image):
    return cv2.normalize(image, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

X_train = np.array([normalize_img(image) for image in X_train], dtype=np.float32)
X_val = np.array([normalize_img(image) for image in X_val], dtype=np.float32)

layer1_depth = 32
filter_size = 5
num_classes = len(np.unique(y_train))
num_neurons = 128

model = Sequential()
model.add(Convolution2D(layer1_depth, filter_size, filter_size, border_mode='valid', input_shape = (125, 200, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(num_neurons))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 121, 196, 32)  2432        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 121, 196, 32)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 60, 98, 32)    0           activation_1[0][0]               
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 60, 98, 32)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [22]:
# TODO: Compile and train the model here.

y_train = np.array(y_train).astype(int)
y_val = np.array(y_val).astype(int)


y_train = np_utils.to_categorical(y_train, nb_classes=num_classes)
y_val = np_utils.to_categorical(y_val, nb_classes=num_classes)

X_train = X_train.reshape(-1, 125, 200, 3)
X_val = X_val.reshape(-1, 125, 200, 3)

nb_epoch = 2
batch_size = 32

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))

Train on 2318 samples, validate on 410 samples
Epoch 1/2
2318/2318 [==============================] - 177s - loss: 8.7081 - acc: 0.4435 - val_loss: 8.4129 - val_acc: 0.4780
Epoch 2/2
2318/2318 [==============================] - 172s - loss: 8.8378 - acc: 0.4517 - val_loss: 8.4129 - val_acc: 0.4780


In [23]:
print( "The validation accuracy is: %.3f" % history.history['val_acc'][-1] )

The validation accuracy is: 0.478


### Testing
Once picked out the best model, it's time to test it.
Load up the test data and use the evaluate() method to see how well it does.

In [34]:
def normalize_img(image):
    return cv2.normalize(image, None, alpha=-0.5, beta=0.5, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

X_test = np.array([normalize_img(image) for image in X_test], dtype=np.float32)
y_test =  np.array(y_test).astype(int)
y_test = np_utils.to_categorical(y_test, nb_classes=num_classes)

X_test = X_test.reshape(-1, 125, 200, 3)

# TODO: Evaluate model on test data
model.evaluate(X_test, y_test)

ValueError: invalid literal for int() with base 10: 'img_00005.jpg'